# Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
from IPython.display import display
import matplotlib.pyplot as plt

# Load data

In [3]:
# VARIABLES:
PATH_metadata = "F:\\Stage\\ePODIUM\\Data\\metadata\\"
PATH_excelFiles = "F:\\Stage\\ePODIUM\\Data\\metadata\\"
MakeExcelFiles = True # Option to put .txt files into Excel files

def loadmetadata(filename):
    originalPATH = PATH_metadata + filename + '.txt'    
    if os.path.exists(originalPATH):  
        metadata = pandas.read_table(originalPATH)
        if(MakeExcelFiles):
            excelPATH = PATH_excelFiles + filename + '.xlsx'       
            metadata.to_excel(excelPATH)            
        return metadata            
    else: 
        print("PATH NOT FOUND:  " + originalPATH) 
        return None

cdi = loadmetadata("cdi")
children = loadmetadata("children")
parents = loadmetadata("parents")

# Process Sound Event Codes file
CODES_overview = loadmetadata("CODES_overview")

# cdi
# children
parents
#CODES_overview

,child,emt_mother,klepel_mother,vc_mother,dyslexia_mother_accToMother,emt_father,klepel_father,vc_father,dyslexia_father_accToFather
0,101,105,101,19,Nee,80,64,10,Ja
1,102,116,112,24,Nee,116,94,24,Nee
2,103,90,100,22,Nee,81,65,21,Ja
3,104,81,54,19,Ja,101,77,24,Nee
4,105,101,79,22,Ja,116,101,23,Nee
...,...,...,...,...,...,...,...,...,...
72,173,96,107,23,Nee,?,?,?,?
73,174,?,?,?,?,82,81,18,Nee
74,175,116,108,22,Nee,?,?,?,?
75,176,114,112,21,Nee,?,?,?,?


In [ ]:
EEG = []
EEG_filenames = []
EEG_filelocation = []

for root, dirs, files in os.walk("F:\\Stage\\ePODIUM\\Data\\not_zip\\"):
    for file in files:
        if file.endswith('.bdf') and len(file) == 8:
            BdfFile = mne.io.read_raw_bdf(root+'\\'+file)
            EEG.append(BdfFile)              
            EEG_filenames.append(file.replace(".bdf", ""))
            EEG_filelocation.append(root+'\\'+file)
            #clear_output(wait=True)            
print(len(EEG), "EEG files loaded")

eeginfo = True
if(eeginfo):
    index = 7
    print('File Location: ',EEG_filelocation[index],'\n')
    print(EEG[index],'\n')
    print(EEG[index].info)

# Data orientation

In [ ]:
index = 8 #EEG data on a single trial of a subject
n_time_samps = EEG[index].n_times
time_secs = EEG[index].times
ch_names = EEG[index].ch_names
n_chan = len(ch_names)  
print('the (cropped) sample data object has {} time samples and {} channels.'
      ''.format(n_time_samps, n_chan))
print('The last time sample is at {} seconds.'.format(time_secs[-1]))
print('The first few channel names are {}.'.format(', '.join(ch_names[:3])))
print()  # insert a blank line in the output

# some examples of raw.info:
print('bad channels:', EEG[index].info['bads'])  # chs marked "bad" during acquisition
print(EEG[index].info['sfreq'], 'Hz')            # sampling frequency
print(EEG[index].info['description'], '\n')      # miscellaneous acquisition info

print(EEG[index].info)

In [ ]:
#extracting data by time index for the first channel
sampling_freq = EEG[index].info['sfreq']
start_stop_seconds = np.array([11, 12])
start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
channel_index = 6
eeg_selection = EEG[index][channel_index, start_sample:stop_sample]
print(eeg_selection)

In [ ]:
x = eeg_selection[1]
y = eeg_selection[0].T
plt.plot(x, y)

In [ ]:
EEG[index].ch_names

In [ ]:
#extract different channels
channel_names = ['Cz', 'Pz']
two_ch = EEG[index][channel_names, start_sample:stop_sample]
#y_offset = np.array([5e-20,0])  
x = two_ch[1]
y = two_ch[0].T # + y_offset
lines = plt.plot(x, y)
plt.legend(lines, channel_names)


# Events

In [ ]:
events = [] 
events_12 = []
for i in range(len(EEG)):
    events.append(numpy.loadtxt("./Event Markers\\" + EEG_filenames[i] + ".txt", dtype = int))        
print(len(events), "Event Marker files loaded")

eventinfo = True
participantIndex = 5
eventIndex = 500
if(SHOW_EVENT_INFO):
    event_time = Events[participantIndex][eventIndex][0]
    event_ID = Events[participantIndex][eventIndex][2]    
    print("\nParticipant", participantIndex,"heard event ID:", event_ID, 
          "after", event_time/EEG[participantIndex].info['sfreq'], 'seconds', 
          'as the ' + str(eventIndex) + "'th event")    
    print("\nEvent ID:", event_ID, "\nCondition:", CODES_overview["condition"][event_ID], "\nStimulus Type:",
         CODES_overview["stimulusType"][event_ID], "\nToken nr:", CODES_overview["nrOfToken"][event_ID],"\n") 

In [ ]:
events = mne.find_events(EEG[index], stim_channel='Fz')